### Imports

In [1]:
%pylab inline
from glob import glob
from nipype.interfaces import afni
from nipype.interfaces.fsl import Level1Design, FEATModel, FILMGLS, FEAT, maths
from nipype.algorithms.modelgen import SpecifyModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node
from os import makedirs
from os.path import abspath, join
import pandas as pd
import shutil


Populating the interactive namespace from numpy and matplotlib


### Experiment Variables

In [2]:
experiment_dir = abspath('.')
output_dir = 'datasink'
working_dir = 'workingdir'

# list of subject identifiers
subject_list = ['sub-s358']

# list of task identifiers
task_list = ['ANT', 'DPX', 'stroop', 'twoByTwo']
task_list = ['WATT3']

# TR of functional images
TR = .68


# Set up Nodes

### Define helper functions

In [3]:
# helper function to create bunch
def subjectinfo(subject_id, task, inspect_inputs=False):
    
    from glob import glob
    import numpy as np
    import pandas as pd
    from os.path import join
    from nipype.interfaces.base import Bunch
    from utils.utils import get_contrasts, parse_EVs, process_confounds
    
    base_dir = '/home/jovyan/work/output'
    
    # strip "sub" from beginning of subject_id if provided
    subject_id = subject_id.replace('sub-','')
    
    ## Get the Events File
    
    # Read the TSV file and convert to pandas dataframe
    event_file = glob(join(base_dir,
                           'Data',
                           'sub-%s' % subject_id,
                           '*', 'func',
                           '*%s*events.tsv' % task))[0]
    events_df = pd.read_csv(event_file,sep = '\t')

    ## Get the Confounds File (output of fmriprep)
    # Read the TSV file and convert to pandas dataframe
    confounds_file = glob(join(base_dir,
                               'Data',
                               'sub-%s' % subject_id,
                               '*', 'func',
                               '*%s*confounds.tsv' % task))[0]
    regressors, regressor_names = process_confounds(confounds_file)
    
    # set up contrasts
    conditions, onsets, durations, amplitudes = parse_EVs(events_df,task)
    
    subjectinfo = Bunch(conditions=conditions,
                        onsets=onsets,
                         durations=durations,
                         amplitudes=amplitudes,
                         tmod=None,
                         pmod=None,
                         regressor_names=regressor_names,
                         regressors=regressors.T.tolist())
    if inspect_inputs==True:
        regressors_df = pd.DataFrame(regressors, columns = regressor_names)
        return events_df, regressors_df
    else:
        contrasts = get_contrasts(task)
        return subjectinfo, contrasts  # this output will later be returned to infosource

def save_subjectinfo(base_directory, subject_id, task, subject_info, contrasts):
    import errno
    from os import makedirs
    from os.path import join
    import cPickle
    task_dir = join(base_directory, '1stLevel', subject_id + '_task_' + task)
    try: 
        makedirs(task_dir)
    except OSError as e:
        if e.errno == errno.EEXIST:
            pass
        else:
            raise
    subjectinfo_path = join(task_dir,'subjectinfo.pkl')
    cPickle.dump(subject_info, open(subjectinfo_path,'w'))
    
    contrast_path = join(task_dir,'contrasts.pkl')
    cPickle.dump(contrasts, open(contrast_path,'w'))
    return (subjectinfo_path, contrast_path)

View one events file used in subject info

In [4]:
bunch, contrasts = subjectinfo('s358','WATT3')
events_df,confounds_df = subjectinfo('s358','WATT3',True)
events_df.head() 

,onset,duration,block_duration,planning,condition,experiment_exp_id,key_press,problem_id,worker_id
0,3.008,5.1102,4737.0,1,UA_with_intermeidate,ward_and_allport,71,0.0,s358
1,7.746,5.7310,1603.0,0,UA_with_intermeidate,ward_and_allport,82,0.0,s358
2,14.724,5.1102,8030.0,1,UA_with_intermeidate,ward_and_allport,89,1.0,s358
3,22.755,5.7310,547.0,0,UA_with_intermeidate,ward_and_allport,82,1.0,s358
4,27.620,5.1102,4796.0,1,UA_without_intermeidate,ward_and_allport,82,2.0,s358


### Specify Input and Output Stream

In [5]:
# Get Subject Info - get subject specific condition information
getsubjectinfo = Node(Function(input_names=['subject_id', 'task'],
                               output_names=['subject_info', 'contrasts'],
                               function=subjectinfo),
                      name='getsubjectinfo')

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'task',
                                            'contrasts']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task', task_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
templates = {'func': join('Data', '{subject_id}','*','func',
                         '*{task}*preproc.nii.gz'),
            'mask': join('Data', '{subject_id}','*','func',
                         '*{task}*brainmask.nii.gz')}
selectfiles = Node(SelectFiles(templates,
                               base_directory = experiment_dir,
                               sort_filelist=True),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory = experiment_dir,
                         container=output_dir),
                name="datasink")

# Save python objects that aren't accomodated by datasink nodes
save_subjectinfo = Node(Function(input_names=['base_directory','subject_id',
                                              'task','subject_info','contrasts'],
                                 output_names=['output_path'],
                                function=save_subjectinfo),
                       name="savesubjectinfo")
save_subjectinfo.inputs.base_directory = join(experiment_dir,output_dir)
# Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                ('fstat', 'FSTST'),
                ('run0.mat', 'designfile.mat')]
datasink.inputs.substitutions = substitutions

### Model Specification

In [6]:
# mask and blur
masker = Node(maths.ApplyMask(),name='masker')

# SpecifyModel - Generates FSL-specific Model
modelspec = Node(SpecifyModel(input_units='secs',
                              time_repetition=TR,
                              high_pass_filter_cutoff=80),
                 name="modelspec")

# Level1Design - Generates an FSL design matrix
level1design = Node(Level1Design(bases={'dgamma':{'derivs': True}},
                                 interscan_interval=TR,
                                 model_serial_correlations=True),
                    name="level1design")

# FEATmodel
level1model = Node(FEATModel(), name="FEATModel")

# FILMGLs
# smooth_autocorr, check default, use FSL default
filmgls = Node(FILMGLS(), name="FILMGLS")

# Run as separate nodes

Useful for debugging

# Workflow

In [7]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = join(experiment_dir, working_dir)

# Connect up the 1st-level analysis components
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('task', 'task')]),
                    (infosource, getsubjectinfo, [('subject_id','subject_id'),
                                                 ('task', 'task')]),
                    (selectfiles, masker, [('func','in_file'),
                                           ('mask', 'mask_file')]),
                    (getsubjectinfo, modelspec, [('subject_info','subject_info')]),
                    (masker, modelspec, [('out_file', 'functional_runs')]),
                    (modelspec, level1design, [('session_info','session_info')]),
                    (getsubjectinfo, level1design, [('contrasts','contrasts')]),
                    (level1design, level1model, [('ev_files', 'ev_files'),
                                                 ('fsf_files','fsf_file')]),
                    (masker, filmgls, [('out_file', 'in_file')]),
                    (level1model, filmgls, [('design_file', 'design_file'),
                                            ('con_file', 'tcon_file'),
                                            ('fcon_file', 'fcon_file')]),
                    (level1model, datasink, [('design_file', '1stLevel.@design_file')]),
                    (filmgls, datasink, [('zstats', '1stLevel.@Z'),
                                        ('fstats', '1stLevel.@F'),
                                        ('tstats','1stLevel.@T'),
                                        ('param_estimates','1stLevel.param_estimates')]),
                    (infosource, save_subjectinfo, [('subject_id','subject_id'),
                                                     ('task', 'task')]),
                    (getsubjectinfo, save_subjectinfo, [('subject_info','subject_info'),
                                                        ('contrasts','contrasts')]),
                    
                    ])


### Run the Workflow


In [12]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 4})

170531-17:45:42,381 workflow INFO:
	 Workflow l1analysis settings: ['check', 'execution', 'logging']
170531-17:45:42,393 workflow INFO:
	 Running in parallel.
170531-17:45:42,396 workflow INFO:
	 Executing: getsubjectinfo.a0 ID: 0
170531-17:45:42,397 workflow INFO:
	 [Job finished] jobname: getsubjectinfo.a0 jobid: 0
170531-17:45:42,399 workflow INFO:
	 Executing: selectfiles.a0 ID: 1
170531-17:45:42,403 workflow INFO:
	 Executing node selectfiles.a0 in dir: /home/jovyan/work/output/workingdir/l1analysis/_subject_id_sub-s358_task_WATT3/selectfiles
170531-17:45:42,433 workflow INFO:
	 [Job finished] jobname: selectfiles.a0 jobid: 1
170531-17:45:42,435 workflow INFO:
	 Executing: masker.a0 ID: 2
170531-17:45:42,439 workflow INFO:
	 [Job finished] jobname: masker.a0 jobid: 2
170531-17:45:42,440 workflow INFO:
	 Executing: savesubjectinfo.a0 ID: 5
170531-17:45:42,740 workflow INFO:
	 [Job finished] jobname: savesubjectinfo.a0 jobid: 5
170531-17:45:42,742 workflow INFO:
	 Executing: modelsp

RuntimeError: Workflow did not execute cleanly. Check log for details

### Visualize Workflow

In [ ]:
# Create 1st-level analysis output graph
l1analysis.write_graph(graph2use='colored', format='png', simple_form=True)
# Visualize the graph
from IPython.display import Image
graph_file=join(l1analysis.base_dir, 'l1analysis', 'graph.dot.png')
Image(graph_file)


In [9]:
try:
    shutil.move(graph_file, 'l1analysis')
except shutil.Error:
    pass
# remove working directory
shutil.rmtree(working_dir)

NameError: name 'graph_file' is not defined

### Visualize Design Matrix

In [ ]:
!tree datasink/1stLevel/sub*


In [ ]:
design_file = glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_ANT','designfile.mat'))[0]
desmtx=numpy.loadtxt(design_file, skiprows=5)
plt.imshow(desmtx,aspect='auto',interpolation='nearest',cmap='gray')
plt.title('Design Matrix')
plt.show()
cc=numpy.corrcoef(desmtx.T)
plt.imshow(cc,aspect='auto',interpolation='nearest', cmap=plt.cm.viridis)
plt.colorbar()
plt.title('Correlation of Regressors')
plt.show()
"""
# show first regressors
plt.figure(figsize=[20,8])
plt.plot(desmtx[:,0],'b', label=subject_info.conditions[0])
plt.plot(desmtx[:,2],'r', label=subject_info.conditions[1])
for i in subject_info.onsets[0]:
    plt.axvline(i/.68, ymin=0, ymax=.1)
for i in subject_info.onsets[1]:
    plt.axvline(i/.68, ymin=0, ymax=.1, c='r')
plt.legend()
plt.title('Congruent/Incongruent Regressors')
"""

### Visualize Results

In [ ]:
from nilearn.plotting import plot_stat_map
from nilearn.image import smooth_img

anatimg = glob(join(experiment_dir,'Data','sub-s358','anat','*T1w*MNI*preproc*'))[0]
contrast_img = glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_ANT','zstat3.nii.gz'))[0]
plot_stat_map(smooth_img(contrast_img, 8), title='stroop effect',
              bg_img=anatimg, threshold=1, display_mode='z', cut_coords=(-30, -15, 0, 15, 30), dim=-1)

In [ ]:
import nilearn.plotting
import nilearn.image
contrast_imgs = sort(glob(join(experiment_dir,'datasink','1stLevel','sub-s358_task_ANT','zstat*.nii.gz')))
for contrast_img in contrast_imgs:
    nilearn.plotting.plot_glass_brain(nilearn.image.smooth_img(contrast_img, 8),
                                      display_mode='lyrz', colorbar=True, plot_abs=False, threshold=0)